In [1]:
import pandas as pd
import numpy as np
!pip install tabulate
from tabulate import tabulate

In [2]:
import itertools
l_permutation = list(itertools.product([1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2]))
df_permutation = pd.DataFrame(l_permutation,columns = ['beta','mu','p','delta','alpha','g_L','g_Z','g_Q','N_bar'])

In [3]:
# Settings
show_decomposition_table = 1
write_decomposition_table_latex = 1

In [4]:
class par: None
class moms: None
show_moments_table = 0
write_moments_table_latex = 0
%run ./main.ipynb

Stored 'start' (str)
Stored 'end' (str)
Stored 'df_estimates' (DataFrame)
Stored 'start' (str)
Stored 'end' (str)
Stored 'df_estimates' (DataFrame)
\begin{tabular}{lcccc}
\toprule
& & & \textit{Estimates} & \\ \cmidrule(lr){3-5}
Parameter name & Symbol & 1984 - 2000 & 2001 - 2016 & Difference \\
\midrule
 Discount factor            & $\beta$   & $0.953$ & $0.968$ & $0.015$  \\
 Mark-up                    & $\mu$     & $1.147$ & $1.172$ & $0.026$  \\
 Disaster probability       & $p$       & $0.030$ & $0.075$ & $0.045$  \\
 Depreciation, pct.         & $\delta$  & $3.085$ & $3.907$ & $0.822$  \\
 Cobb-Douglas parameter     & $\alpha$  & $0.357$ & $0.300$ & $-0.057$ \\
 Population growth, pct.    & $g_L$     & $0.263$ & $0.682$ & $0.419$  \\
 TFP growth, pct.           & $g_Z$     & $1.678$ & $0.715$ & $-0.963$ \\
 Technological change, pct. & $g_Q$     & $0.678$ & $0.411$ & $-0.267$ \\
 Labour Supply              & $\bar{N}$ & $0.489$ & $0.515$ & $0.026$  \\
\bottomrule
\end{tabular}
St

In [5]:
#Assumptions and settings
par.sigma = 0.5
par.theta = 12
par.b = -np.log(1-0.15)
par.bh = -np.log((2-np.exp(-par.b)))

#Define equations
def growthpop(par,moms):
    return par.g_L

def priceinvt(par,moms):
    return -par.g_Q

def ik(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta)

def EmpPop(par,moms):
    return par.N_bar

def ProfitY(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)

def TFPgrowth(par,moms):
    moms.ProfitY = (par.mu+par.alpha-1)/(par.mu)
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    return par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q)

def ProfitK(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma)*MOM
    r_star = 1/beta_star - 1
    return ((par.mu + par.alpha -1)/(par.alpha))*(r_star + par.delta + par.g_Q*(1+r_star))

def RF(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    MOM2 = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))
    MOM3 = ((1-2*par.p)+par.p*np.exp(-par.bh*(-par.theta)) + par.p*np.exp(-par.b*(-par.theta))) 
    return MOM2/(MOM3*beta_star)-1

def PD(par,moms):
    par.b = -np.log(1-0.15)
    par.bh = -np.log((2-np.exp(-par.b)))
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta))     
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    r_star = 1/beta_star - 1
    #return ((1+par.g_T)/(r_star-par.g_T))
    return (beta_star*(1+par.g_T)/(1-beta_star*(1+par.g_T)))
    
result = [(ProfitK(par,moms),ProfitY(par,moms),RF(par,moms), growthpop(par,moms), priceinvt(par,moms),PD(par,moms), TFPgrowth(par,moms), ik(par,moms), EmpPop(par,moms))
            for par.beta in [estimates.loc["beta","1984 - 2000"],estimates.loc["beta","2001 - 2016"]]
            for par.mu in [estimates.loc["mu","1984 - 2000"],estimates.loc["mu","2001 - 2016"]]
            for par.p in [estimates.loc["p","1984 - 2000"],estimates.loc["p","2001 - 2016"]]
            for par.delta in [estimates.loc["delta","1984 - 2000"],estimates.loc["delta","2001 - 2016"]]
            for par.alpha in [estimates.loc["alpha","1984 - 2000"],estimates.loc["alpha","2001 - 2016"]]
            for par.g_L in [estimates.loc["g_L","1984 - 2000"],estimates.loc["g_L","2001 - 2016"]]
            for par.g_Z in [estimates.loc["g_Z","1984 - 2000"],estimates.loc["g_Z","2001 - 2016"]]
            for par.g_Q in [estimates.loc["g_Q","1984 - 2000"],estimates.loc["g_Q","2001 - 2016"]]
            for par.N_bar in [estimates.loc["N_bar","1984 - 2000"],estimates.loc["N_bar","2001 - 2016"]]
         ]
df = pd.DataFrame(result, columns =["ProfitK","ProfitY","RF", "growthpop","priceinvt","PD","TFPgrowth","ik","EmpPop"])

In [6]:
df = df_permutation.join(df)

In [22]:
moments = ["ProfitK","ProfitY","RF","PD","ik","TFPgrowth","priceinvt","growthpop","EmpPop"]
parameters = ["beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]

df_results = pd.DataFrame(columns=parameters)
df_results["moment"] = moments
df_results = df_results.set_index("moment",drop=True)

for mom in moments:
    for parm in parameters:
        result = df.loc[(df[parm] == 2),mom].mean()-df.loc[(df[parm] == 1),mom].mean() 
        df_results.loc[mom,parm] = result    


df_results_formatted = df_results.copy()        
df_results_formatted = df_results_formatted.astype(float) # Convert all to floats (they appear to be strings?)
df_results_formatted['change'] = df_results_formatted.sum(axis=1, skipna=True)    # Sum rows  
df_results_formatted['p2'] = [moms.ProfitK,moms.s_K,moms.RF,moms.PD,moms.ik,moms.TFPgrowth,moms.g_Q,moms.g_L,moms.N_bar]
df_results_formatted['p1'] = df_results_formatted['p2']-df_results_formatted['change']
df_results_formatted = df_results_formatted[["p1","p2","change","beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]] # Change order – sum first

scalar = [1 if i=="PD" else 100 for i in df_results_formatted.index.tolist()]  #Create list, 1 for PD, 100 for all other
df_results_formatted = df_results_formatted.multiply(scalar,axis=0)

for var in df_results_formatted.columns.tolist()[:]:
    df_results_formatted[var] = df_results_formatted[var].map('${:,.2f}$'.format)

if show_decomposition_table == 1:
    display(df_results_formatted)
    
if write_decomposition_table_latex == 1:
    latex = tabulate(df_results_formatted, showindex=True, tablefmt="latex_raw")
    
    print("\\begin{tabular}{lrrrrrrrrrrrr}")
    print("\\toprule")
    print(" & \multicolumn{3}{c}{\\textit{Data}} & \multicolumn{9}{c}{\\textit{Decomposition of $\\Delta$}}  \\\\ \\cmidrule(lr){2-4} \\cmidrule(lr){5-13}")
    print(" & P1 & P2 & $\\Delta$ & $\\beta$ & $\\mu$ & $p$ & $\\delta$ & $\\alpha$ & $g_L$ & $g_Z$ & $g_Q$ & $\\bar{N}$   \\\\")
    print("\\midrule")
    print(latex[38:-21])
    print("\\bottomrule")
    print("\\end{tabular}")

,p1,p2,change,beta,mu,p,delta,alpha,g_L,g_Z,g_Q,N_bar
moment,,,,,,,,,,,,
ProfitK,$15.26$,$15.00$,$-0.26$,$-2.44$,$0.80$,$1.12$,$1.22$,$0.68$,$-0.00$,$-1.13$,$-0.52$,$0.00$
ProfitY,$43.95$,$40.33$,$-3.62$,$0.00$,$1.29$,$0.00$,$0.00$,$-4.91$,$0.00$,$0.00$,$0.00$,$0.00$
RF,$4.20$,$-0.60$,$-4.80$,$-1.56$,$0.00$,$-2.34$,$0.00$,$-0.11$,$-0.00$,$-0.72$,$-0.07$,$0.00$
PD,$28.41$,$30.86$,$2.45$,$15.50$,$0.00$,$-7.80$,$0.00$,$-1.35$,$4.72$,$-7.86$,$-0.75$,$0.00$
ik,$7.07$,$6.22$,$-0.85$,$0.00$,$0.00$,$0.00$,$0.82$,$-0.23$,$0.43$,$-1.46$,$-0.41$,$0.00$
TFPgrowth,$1.39$,$0.56$,$-0.84$,$0.00$,$-0.03$,$0.00$,$0.00$,$-0.00$,$0.00$,$-0.84$,$0.04$,$0.00$
priceinvt,$0.14$,$0.41$,$0.27$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.27$,$0.00$
growthpop,$0.26$,$0.68$,$0.42$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.42$,$0.00$,$0.00$,$0.00$
EmpPop,$48.90$,$51.48$,$2.58$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$0.00$,$2.58$


\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & \multicolumn{3}{c}{\textit{Data}} & \multicolumn{9}{c}{\textit{Decomposition of $\Delta$}}  \\ \cmidrule(lr){2-4} \cmidrule(lr){5-13}
 & P1 & P2 & $\Delta$ & $\beta$ & $\mu$ & $p$ & $\delta$ & $\alpha$ & $g_L$ & $g_Z$ & $g_Q$ & $\bar{N}$   \\
\midrule
 ProfitK   & $15.26$ & $15.00$ & $-0.26$ & $-2.44$ & $0.80$  & $1.12$  & $1.22$ & $0.68$  & $-0.00$ & $-1.13$ & $-0.52$ & $0.00$ \\
 ProfitY   & $43.95$ & $40.33$ & $-3.62$ & $0.00$  & $1.29$  & $0.00$  & $0.00$ & $-4.91$ & $0.00$  & $0.00$  & $0.00$  & $0.00$ \\
 RF        & $4.20$  & $-0.60$ & $-4.80$ & $-1.56$ & $0.00$  & $-2.34$ & $0.00$ & $-0.11$ & $-0.00$ & $-0.72$ & $-0.07$ & $0.00$ \\
 PD        & $28.41$ & $30.86$ & $2.45$  & $15.50$ & $0.00$  & $-7.80$ & $0.00$ & $-1.35$ & $4.72$  & $-7.86$ & $-0.75$ & $0.00$ \\
 ik        & $7.07$  & $6.22$  & $-0.85$ & $0.00$  & $0.00$  & $0.00$  & $0.82$ & $-0.23$ & $0.43$  & $-1.46$ & $-0.41$ & $0.00$ \\
 TFPgrowth & $1.39$  & $0.56$  & $-0.84$ & $0